In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random
import pandas as pd

In [2]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

In [3]:
list_Validation = []
user_list = []
book_list = []
r_list = []
list_Train = []
for user,book,r in readCSV("train_Interactions.csv.gz"):
    list_Validation.append([user, book])
    user_list.append(user)
    book_list.append(book)
    r_list.append(r)
    list_Train.append([user, book])
    
validate_book = book_list[190000:]
validate_user = user_list[190000:]
train_user = user_list[:189999]
train_book = book_list[:189999]
list_Train = list_Train[:189999]
list_Validation = list_Validation[190000:]
list_valiY = [1]*len(list_Validation)

for i in validate_user:
    randomBook = random.choice(validate_book)
    randomlist = [i, randomBook]
    while randomlist in list_Validation:
        randomBook = random.choice(validate_book)
        randomlist = [i, randomBook]
    list_Validation.append(randomlist)
    list_valiY.append(0)

In [5]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)
    
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    
# Generate a negative set

userSet = set()
bookSet = set()
readSet = set()

for u,b,r in allRatings:
    userSet.add(u)
    bookSet.add(b)
    readSet.add((u,b))

lUserSet = list(userSet)
lBookSet = list(bookSet)

notRead = set()
for u,b,r in ratingsValid:
    #u = random.choice(lUserSet)
    b = random.choice(lBookSet)
    if (u,b) in readSet or (u,b) in notRead: continue
    notRead.add((u,b))

readValid = set()
for u,b,r in ratingsValid:
    readValid.add((u,b))
    
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

correct = 0
p0, p1 = 0,0


for (label,sample) in [(1, readValid), (0, notRead)]:
    for (u,b) in sample:
        maxSim = 0
        users = set(ratingsPerItem[b])
        for b2,_ in ratingsPerUser[u]:
            sim = Jaccard(users,set(ratingsPerItem[b2]))
            if sim > maxSim:
                maxSim = sim
        pred = 0
        
        if maxSim > 0.05 or len(ratingsPerItem[b]) > 40:
            pred = 1
            p1 += 1
        else:
            p0 += 1
        if pred == label:
            correct += 1
            

In [ ]:
matrix = []

bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: 
        break


for l in readCSV("train_Interactions.csv.gz"):
    sub_matrix = []
    #if l.startswith("userID"):
    #header
        #predictions.write(l)
        #continue
    u,b = l.strip().split('-')
    maxSim = 0
    users = set(ratingsPerItem[b])
    if b in return1:
        sub_matrix.append(int(1))
        sub_matrix.append(int(1))
    else:
        sub_matrix.append(int(1))
        sub_matrix.append(int(0))
        
    for b2,_ in ratingsPerUser[u]:
        sim = Jaccard(users,set(ratingsPerItem[b2]))
        if sim > maxSim:
            maxSim = sim
    pred = 0
    if maxSim > 0.013 or len(ratingsPerItem[b]) > 40:
        pred = 1
    sub_matrix.append(int(pred))
   # _ = predictions.write(u + '-' + b + ',' + str(pred) + '\n')

    matrix.append(sub_matrix)






    
    




In [ ]:
print(matrix)

In [ ]:
y_train = [1]*(len(matrix)-1)
y_train.append(0)

In [ ]:
correct = 0

for i, j in zip(open("assipredictions_Read.txt"), list_valiY):
    if i[20] is 't':
        continue
    else:
        if int(i[20]) == int(j):
            correct = correct +  1
accuracy = correct/len(list_valiY)
print(accuracy)

In [ ]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(matrix, y_train)

In [ ]:
pred = mod.predict(matrix)

In [ ]:
correct = pred == list_valiY 
sum(correct) / len(correct)

In [ ]:
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    if b in return1:
        predictions.write(u + '-' + b + ",1\n")
    else:
        predictions.write(u + '-' + b + ",0\n")
predictions.close()